In [46]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [47]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_openai import ChatOpenAI


In [48]:
file = 'report_225.csv'
loader = CSVLoader(file_path=file)

In [49]:
from langchain.indexes import VectorstoreIndexCreator

In [50]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [51]:
query ="Please give me only name , role and score  \
         in a table in  markdown"       

In [52]:
llm_replacement_model = ChatOpenAI(temperature=0, 
                               model='gpt-4-0125-preview')

response = index.query(query, 
                       llm = llm_replacement_model)

In [53]:
display(Markdown(response))

```markdown
| Name              | Role                           | Score |
|-------------------|--------------------------------|-------|
| Praveen Tadikemalla | Technical Team Lead/Engineering Manager | 56    |
```

step by step

In [54]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [55]:
docs = loader.load()

In [56]:
docs[0]

Document(page_content='role: Technical Team Lead/Engineering Manager\ncandidate: Praveen Tadikemalla\nlocation: Not specified\nscore: 56\nrecommendation: Praveen Tadikemalla demonstrates strong technical expertise, particularly in .NET technologies, which aligns well with the role of Technical Team Lead/Engineering Manager. However, he lacks experience with several key technologies such as React and AWS. With over 15 years of experience, including leadership roles, he has the potential to grow into the role with some additional training in the missing skills.\nSimilar Skills: C#, JavaScript, HTML, DevOps, Node.js\nMissing Skills: React, TypeScript, .NET Core, HTML5, CSS, PostCSS, AppSync, Elasticsearch, AWS, Web APIs, PCI-DSS Security Practices, Auth0, SAML\nPreferable Skills: DevOps\nexperiences: Infosys [Technology Lead (15 years)] ,   Wipro Limited [Senior Consultant (0 years)]\nfilename: Naukri_praveentadikemalla[15y_0m].pdf', metadata={'source': 'resumes_report (3).csv', 'row': 0}

In [57]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [58]:
embed = embeddings.embed_query("Hi my name is kshitij")

In [59]:
print(len(embed))

1536


In [60]:
print(embed[:5])

[-0.012838728974454896, 0.0023732881986838466, -0.022881927411664532, -0.017950924460904436, -0.031786207725038304]


In [61]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [62]:
query = "Please give me score"

In [63]:
docs = db.similarity_search(query)

In [64]:
len(docs)

1

In [65]:
docs[0]

Document(page_content='role: Technical Team Lead/Engineering Manager\ncandidate: Praveen Tadikemalla\nlocation: Not specified\nscore: 56\nrecommendation: Praveen Tadikemalla demonstrates strong technical expertise, particularly in .NET technologies, which aligns well with the role of Technical Team Lead/Engineering Manager. However, he lacks experience with several key technologies such as React and AWS. With over 15 years of experience, including leadership roles, he has the potential to grow into the role with some additional training in the missing skills.\nSimilar Skills: C#, JavaScript, HTML, DevOps, Node.js\nMissing Skills: React, TypeScript, .NET Core, HTML5, CSS, PostCSS, AppSync, Elasticsearch, AWS, Web APIs, PCI-DSS Security Practices, Auth0, SAML\nPreferable Skills: DevOps\nexperiences: Infosys [Technology Lead (15 years)] ,   Wipro Limited [Senior Consultant (0 years)]\nfilename: Naukri_praveentadikemalla[15y_0m].pdf', metadata={'source': 'resumes_report (3).csv', 'row': 0}

In [66]:
retriever = db.as_retriever()

In [67]:
llm_model = 'gpt-4-0125-preview'

In [68]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [69]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [77]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all skills praveen has.") 


In [78]:
display(Markdown(response))

Based on the information provided, Praveen Tadikemalla has the following skills:

1. C#
2. JavaScript
3. HTML
4. DevOps
5. Node.js

In [79]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [84]:
query =  "Please give me all names"

In [85]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [86]:
display(Markdown(response))

Praveen Tadikemalla

In [76]:
response = index.query(query, llm=llm)

In [83]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])